In [ ]:
#import packages
!pip install pytorchcv
import  pytorchcv
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2


import os
import pandas as pd 
import random
import gc
import torch.cuda as cuda
from google.colab import drive
import torchvision as tv
import torchvision.transforms as transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import ImageFile

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install GPUtil
import GPUtil as GPU
import humanize
import psutil
import os 

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))




In [ ]:
# # PATH="/content/drive/My Drive/dog/ResModel.pth"
# # ResModel = torch.load(PATH)

# PATH="/content/drive/My Drive/dog/IncModel.pth"
# # IncModel = torch.load(PATH)
# IncModel = torch.load(PATH)
# # PATH="/content/drive/My Drive/dog/RsxModel.pth"
# # RsxModel = torch.load(PATH)
# # IncModel.to(torch.float16)

# torch.save(IncModel,"eerer.pth")

In [ ]:


!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
EFNet = EfficientNet.from_pretrained('efficientnet-b3')
# EFNet = models.resnet18(pretrained=True)
print(EFNet)
EFNet._fc = torch.nn.Sequential()#1536
for  name,param in EFNet.named_parameters():
  if ("bn" not in name):
    param.requires_grad = False
EFNet.set_swish(memory_efficient=False)


Loaded pretrained weights for efficientnet-b3
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2

In [ ]:
class mlp(torch.nn.Module):
  def  __init__(self,input_size, output_num):
    super(mlp,self).__init__()
    self.fc1=torch.nn.Linear(input_size,output_num)
 
    self.relu=torch.nn.ReLU()
    self.drop4= torch.nn.Dropout(p=0.8)
    
    
  def forward(self,x):
   
    out=self.drop4(x)
    out=self.fc1(out)

    return out

In [ ]:
model=mlp(1536 , 150)

In [ ]:
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=(400,350), scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        # transforms.CenterCrop(size=(400,300)),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]) 
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=(400,350)),
        # transforms.CenterCrop(size=(400,350)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
data = {
    'train':
    datasets.ImageFolder(root="/content/drive/My Drive/dog/batch1/train_dir/", transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root="/content/drive/My Drive/dog/batch1/valid_dir/", transform=image_transforms['valid']),
    'test':
    datasets.ImageFolder(root="/content/drive/My Drive/dog/test_dir/", transform=image_transforms['valid']),
}


In [ ]:
train_samples_lenght = len(data['train'])
valid_samples_lenght = len(data['valid'])
test_samples_lenght = len(data['test'])

batch_size_train = 25
batch_size_valid = 10
numTrainBatches =  int(np.ceil(train_samples_lenght / batch_size_train))
numValidBatches =  int(np.ceil(valid_samples_lenght / batch_size_valid))
numTestBatches =  int(np.ceil(test_samples_lenght / batch_size_valid))


dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size_train, shuffle=True),
    'val': DataLoader(data['valid'], batch_size=batch_size_valid, shuffle=False),
    'test': DataLoader(data['test'], batch_size=batch_size_valid, shuffle=False)

}

In [ ]:
print(numTrainBatches)
print(numValidBatches)
print(numTestBatches)

588
361
452


In [ ]:
def cal_accuracy(pred_classes,real_classes):#  shape ->  both : [number of test samples,]
  bool_array=(pred_classes==real_classes) # example: bool_array=[True, False, True, False, True]
  True_pred_counts=np.count_nonzero(bool_array) # count number of Trues in [True, False, True, False, True]
  return True_pred_counts/pred_classes.shape[0]

In [ ]:
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), eps=1e-08,lr=0.0001)


In [ ]:
### TEST recall ###

# from efficientnet_pytorch import EfficientNet
# EFNet = EfficientNet.from_pretrained('efficientnet-b2')
# # print(EFNet)
# EFNet._fc = torch.nn.Sequential()#1408
# for  name,param in EFNet.named_parameters():
#   if ("bn" not in name):
#     param.requires_grad = False
# PATH="/content/drive/My Drive/dog/EFN.pth"
# torch.save(EFNet,PATH)

# EFNet = torch.load(PATH)
PATH ="/content/drive/My Drive/dog/EFN.pth"

EFNet = torch.load(PATH)

f = open("breed.txt", "r").readlines()

corr = [0 for i in range(150)]
es = [[0] for i in range(150)]
N = [0 for i in range(150)]
def cal_accuracy2(pred_classes,real_classes):#  shape ->  both : [number of test samples,]
  for j in range(len(pred_classes)):
    N[real_classes[j]] += 1
    if pred_classes[j] == real_classes[j]:
      corr[real_classes[j]] += 1
      es[real_classes[j]].append(pred_classes[j])


  bool_array=(pred_classes==real_classes) # example: bool_array=[True, False, True, False, True]
  True_pred_counts=np.count_nonzero(bool_array) # count number of Trues in [True, False, True, False, True]
  # prin(pred_classes)
  return True_pred_counts/pred_classes.shape[0]





model1=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net1.pth"
checkpoint = torch.load(PATH)
model1.load_state_dict(checkpoint['model_state_dict'])
model1 = model1.cuda()

model2=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net2.pth"
checkpoint = torch.load(PATH)
model2.load_state_dict(checkpoint['model_state_dict'])
model2 = model2.cuda()

model3=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net3.pth"
checkpoint = torch.load(PATH)
model3.load_state_dict(checkpoint['model_state_dict'])
model3 = model3.cuda()


model4=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net4.pth"
checkpoint = torch.load(PATH)
model4.load_state_dict(checkpoint['model_state_dict'])
model4 = model4.cuda()

model5=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net5.pth"
checkpoint = torch.load(PATH)
model5.load_state_dict(checkpoint['model_state_dict'])
model5 = model5.cuda()



test_loss=0
ac = 0 
testiter = iter(dataloaders['test']) 
for k in range(numTestBatches):
    xx, yy = next(testiter) #features: torch.Size( batch_size, 3 ,400, 399) , labels: torch.Size(batch_size)
    

    if cuda.is_available():
          xx = xx.cuda()
          yy = yy.cuda()

          EFNet = EFNet.cuda()
          model1 = model1.cuda()
          model2 = model2.cuda()
          model3 = model3.cuda()
          model4 = model4.cuda()
          model5 = model5.cuda()


    EFNet.eval()
    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    model5.eval()

    xx = EFNet(xx)
    outputss1=model1(xx) # output: torch.Size([batchsize, 150])  | 150 is number of output neurons
    outputss2=model2(xx)
    outputss3=model3(xx)
    outputss4=model4(xx)
    outputss5=model5(xx)
    
    outputss = (outputss1+outputss2+outputss3+outputss4+outputss5)/5

    loss=criterion(outputss,yy)
    test_loss=test_loss+loss.data
    pred_classes=torch.max(outputss.data, 1)[1]
    acc=cal_accuracy2(np.array(pred_classes.cpu()),np.array(yy.cpu()))
    ac=acc+ac


    del outputss
    del loss
    del xx
    del yy
    gc.collect()
    torch.cuda.empty_cache() 
# print("Epoch: ",i+1," Test loss=",test_loss/numTestBatches," Accuracy: ",ac*100 / numTestBatches)1
# print(" Test loss=",test_loss/numTestBatches," Accuracy: ",ac*100 / numTestBatches)
print(N)
corrDict = {}
for index,val in enumerate(corr):
 corrDict[index]=val

srt=sorted(corrDict.items(),key=lambda x:x[1])
for i in srt:
  print("")
  print(i[0],i[1],f[i[0]][:-1],end=" ")
  # for j in (es[i[0]]):
  #   print(j,end=",")
  print((len(es[i[0]])-1)/N[i[0]], end=",")

corrDict2 = {}
for index,val in enumerate(corr):
  corrDict[index]=val

[17, 24, 19, 20, 18, 16, 13, 11, 16, 19, 14, 16, 22, 15, 13, 17, 17, 19, 16, 20, 21, 16, 15, 10, 12, 9, 8, 13, 10, 8, 7, 30, 47, 33, 40, 32, 30, 39, 41, 35, 39, 36, 43, 31, 37, 37, 34, 30, 34, 30, 36, 30, 30, 30, 30, 30, 31, 39, 31, 33, 30, 39, 30, 31, 30, 30, 36, 30, 31, 30, 31, 32, 31, 40, 30, 31, 30, 30, 31, 30, 30, 31, 42, 33, 30, 37, 31, 33, 30, 43, 36, 37, 31, 30, 35, 30, 30, 34, 39, 42, 37, 35, 30, 50, 31, 36, 31, 30, 39, 34, 39, 37, 33, 30, 39, 29, 36, 43, 40, 29, 34, 30, 34, 40, 43, 30, 31, 46, 40, 31, 42, 38, 36, 31, 31, 31, 31, 30, 30, 41, 30, 34, 30, 30, 32, 30, 33, 37, 31, 32]

23 7 Bloodhound 0.7,
26 7 Border_terrier 0.875,
29 7 Boston_terrier 0.875,
30 7 Bouvier_des_Flandres 1.0,
14 8 Beagle 0.6153846153846154,
16 9 Beauceron 0.5294117647058824,
25 9 Border_collie 1.0,
7 10 Anatolian_shepherd_dog 0.9090909090909091,
24 10 Bluetick 0.8333333333333334,
28 10 Boston_bull 1.0,
6 11 American_Staffordshire_terrier 0.8461538461538461,
0 12 Affenpinscher 0.7058823529411765,
9 12

In [ ]:
### TEST  acc ###

# from efficientnet_pytorch import EfficientNet
# EFNet = EfficientNet.from_pretrained('efficientnet-b2')
# # print(EFNet)
# EFNet._fc = torch.nn.Sequential()#1408
# for  name,param in EFNet.named_parameters():
#   if ("bn" not in name):
#     param.requires_grad = False
# PATH="/content/drive/My Drive/dog/EFN.pth"
# torch.save(EFNet,PATH)

# EFNet = torch.load(PATH)
PATH ="/content/drive/My Drive/dog/EFN.pth"

EFNet = torch.load(PATH)

f = open("breed.txt", "r").readlines()
a = []
corr = [0 for i in range(150)]
# es = [[0] for i in range(150)]
N = [0 for i in range(150)]
def cal_accuracy2(pred_classes,real_classes):#  shape ->  both : [number of test samples,]
  for j in range(len(pred_classes)):
    N[pred_classes[j]] += 1
    if pred_classes[j] == real_classes[j]:
      corr[pred_classes[j]] += 1
      # es[pred_classes[j]].append(pred_classes[j])


  bool_array=(pred_classes==real_classes) # example: bool_array=[True, False, True, False, True]
  True_pred_counts=np.count_nonzero(bool_array) # count number of Trues in [True, False, True, False, True]
  return True_pred_counts/pred_classes.shape[0]





model1=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net1.pth"
checkpoint = torch.load(PATH)
model1.load_state_dict(checkpoint['model_state_dict'])
model1 = model1.cuda()

model2=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net2.pth"
checkpoint = torch.load(PATH)
model2.load_state_dict(checkpoint['model_state_dict'])
model2 = model2.cuda()

model3=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net3.pth"
checkpoint = torch.load(PATH)
model3.load_state_dict(checkpoint['model_state_dict'])
model3 = model3.cuda()


model4=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net4.pth"
checkpoint = torch.load(PATH)
model4.load_state_dict(checkpoint['model_state_dict'])
model4 = model4.cuda()

model5=mlp(1536 , 150)
PATH="/content/drive/My Drive/dog/net5.pth"
checkpoint = torch.load(PATH)
model5.load_state_dict(checkpoint['model_state_dict'])
model5 = model5.cuda()



test_loss=0
ac = 0 
testiter = iter(dataloaders['test']) 
for k in range(numTestBatches):
    xx, yy = next(testiter) #features: torch.Size( batch_size, 3 ,400, 399) , labels: torch.Size(batch_size)
    

    if cuda.is_available():
          xx = xx.cuda()
          yy = yy.cuda()

          EFNet = EFNet.cuda()
          model1 = model1.cuda()
          model2 = model2.cuda()
          model3 = model3.cuda()
          model4 = model4.cuda()
          model5 = model5.cuda()


    EFNet.eval()
    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    model5.eval()

    xx = EFNet(xx)
    outputss1=model1(xx) # output: torch.Size([batchsize, 150])  | 150 is number of output neurons
    outputss2=model2(xx)
    outputss3=model3(xx)
    outputss4=model4(xx)
    outputss5=model5(xx)
    
    outputss = (outputss1+outputss2+outputss3+outputss4+outputss5)/5

    loss=criterion(outputss,yy)
    test_loss=test_loss+loss.data
    # print(loss.data,test_loss)
    pred_classes=torch.max(outputss.data, 1)[1]
    acc=cal_accuracy2(np.array(pred_classes.cpu()),np.array(yy.cpu()))
    ac=acc+ac


    del outputss
    del loss
    del xx
    del yy
    gc.collect()
    torch.cuda.empty_cache() 
# print("Epoch: ",i+1," Test loss=",test_loss/numTestBatches," Accuracy: ",ac*100 / numTestBatches)1
print(" Test loss=",test_loss/numTestBatches," Accuracy: ",ac*100 / numTestBatches)
# print(N)
corrDict = {}
for index,val in enumerate(corr):
  corrDict[index]=val

srt=sorted(corrDict.items(),key=lambda x:x[1])
for i in srt:
  print("")
  print(i[0],i[1],f[i[0]][:-1],end=" ")
  # for j in (es[i[0]]):
  #   print(j,end=",")
  # print((len(es[i[0]])-1)/N[i[0]], end=",")
  print(corr[i[0]]/N[i[0]], end=",")
  # a.append(corr[i[0]]/N[i[0]])

corrDict2 = {}
for index,val in enumerate(corr):
  corrDict[index]=val
# print()
# print(np.mean(a))

 Test loss= tensor(0.2281, device='cuda:0')  Accuracy:  92.58849557522109

23 7 Bloodhound 1.0,
26 7 Border_terrier 1.0,
29 7 Boston_terrier 1.0,
30 7 Bouvier_des_Flandres 1.0,
14 8 Beagle 0.7272727272727273,
16 9 Beauceron 0.6428571428571429,
25 9 Border_collie 1.0,
7 10 Anatolian_shepherd_dog 1.0,
24 10 Bluetick 0.7692307692307693,
28 10 Boston_bull 1.0,
6 11 American_Staffordshire_terrier 1.0,
0 12 Affenpinscher 0.8571428571428571,
9 12 Australian_cattle_dog 0.8,
18 12 Belgian_tervuren 1.0,
22 12 Blenheim_spaniel 0.631578947368421,
27 12 Borzoi 0.8571428571428571,
10 13 Australian_shepherd 1.0,
2 14 African_hunting_dog 0.875,
8 14 Appenzeller 0.8235294117647058,
13 14 Basset 1.0,
15 14 Bearded_collie 0.8235294117647058,
3 15 Airedale 0.9375,
5 15 American_Pitbull_Terrier 1.0,
17 15 Bedlington_terrier 1.0,
1 16 Afghan_hound 0.6666666666666666,
11 16 Australian_terrier 1.0,
21 16 Black-and-tan_coonhound 0.9411764705882353,
64 16 English_springer 0.9411764705882353,
4 17 Akita 0.85,
19

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
b = []
for i in range(150):
    b.append(corr[i]/N[i])
np.mean(b)

0.9267696540199156

In [ ]:
if __name__ == "__main__":
    PATH="/content/drive/My Drive/dog/net5.pth"
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    m = mlp(1536 , 150)
    m.load_state_dict(checkpoint['model_state_dict'])
    mlp.__module__ = "run"
    PATH="/content/drive/My Drive/dog/mlp5.pth"
    torch.save({

              'model_state_dict': m.state_dict(),

              }, PATH)

In [ ]:
PATH ="/content/drive/My Drive/dog/EFN.pth"

torch.save(EFNet,PATH)